# Exploratory doc2vec


In [ ]:
import os
import gensim
import numpy as np
import scipy as sp
from smart_open import smart_open
import multiprocessing

cores = multiprocessing.cpu_count()
usecores=np.int(3*cores/4)




Get and process stopwords. List has been gatherd from various sources and expanded to include (some) utterances such as uhm etc.

In [ ]:
with open('/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded.txt','r') as stopwords_in:
    dutch_stopwords =stopwords_in.readlines()
    for i,sw in enumerate(dutch_stopwords):
        dutch_stopwords[i] = sw.replace('\n','')
        
    dutch_stopwords = gensim.utils.simple_preprocess(' '.join(dutch_stopwords),deacc=True)
    #print(dutch_stopwords)

### Define utility functions

In [ ]:
def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

This (below) reads in the corpus in unprocessed fashin allowing inspection for relevance 

In [ ]:
def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1
        

In [ ]:
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output/TargetSize150'
#corpus_text_file = 'test_GV_corpus_terms_para.txt'
#corpus_filenames = 'test_GV_corpus_terms_para_filenames.txt'
corpus_token_file = 'GV_corpus_terms_para_150.txt'
corpus_text_file = 'GV_corpus_terms_para_150.txt'
corpus_filenames = 'GV_corpus_terms_para_150_filenames.txt'
full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames) 

In [ ]:
corp = list(read_corpus(full_ctokf,labeled=True))

In [ ]:
corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

In [ ]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=50, negative=15, hs=0, min_count=4, epochs=30, workers=usecores)

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=30)#, workers=usecores)



In [ ]:
model.build_vocab(corp)

In [ ]:
%time model.train(corp, total_examples=model.corpus_count, epochs=model.epochs)

load or save as desired

In [ ]:
#model.save('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30_freeze.d2v')
model = gensim.utils.SaveLoad.load('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30_freeze.d2v')




This is an initial test of retrieval on the full corpus

In [ ]:
ranks = []
first_rank =[]
second_ranks = []
least_ranks =[]
doc_id =0
#for i in range(len(corp)):
for i in range(100):
    inferred_vector = model.infer_vector(corp[i].words)
    inferred_vector_tag = corp[i].tags
    print('actual snippet')
    print(corp[i].words)
    print(corp[i].tags)
    print("-------------")
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [tag for tag,sim in sims].index(inferred_vector_tag)
    ranks.append(rank)
    second_ranks.append(sims[1])
    least_ranks.append(sims[-1])
    print(corp_lookup[sims[0][0]])
    print("++++")
    print(rank)
    print(corp_lookup[inferred_vector_tag[0]])
    #print(sims[1])
    #print(sims[2])
    #print(sims[-1])
    print('=======================')
    print
#sims[0]
    #rank = [tag for tag, sim in sims].index(inferred_vector_tag)
    #ranks.append(rank)
    #second_ranks.append(sims[1])
    #least_ranks.append[sims[-1]]

In [ ]:
ranks



### Test fragments 1 & 2 are self-composed 3 is an actual fragmnent from the corpus

In [ ]:
test_term = 'hij sloeg er op dat het kapot ging en schoot met zijn geweer. het was verschrikkelijk om aan te zien. ik verstopte me, maar kon het geluid van de explosies niet uit mijn ooren krijgen. Toen het over was ben ik zo snel ik kon gaan rennen.'
test_term_2 = 'Toen waren wij dus in Auschwitz. En toen? Nou, direkt naar een of ander gebouw toe. Uitkleden en de gevangenis kleding aan doen. Ik zag ook meteen al hoe een jongetje van misschien 4 of 5 van zijn moeder weg getrokken werd door een officier van de SS. En maar huilen en gillen.'#' Totdat een schot viel en alles stil was.'
test_term_3 = "Een razzia . Dus die ... . 'Ja , wat doen we ? ' . En we lopen over het dak , en toen hoorden we vanaf de graansilo , daar stond een luchtafweergeschut , en daar kregen we vuur van . Toen vlogen de kogels om onze oren . Toen zij we als de sodomieter via de schoorsteen , in dekking naar Dordtselaan 98 gevlucht . En daar kwamen we en ja , wat doen we ? Ja , wij gaan niet naar beneden toe . Dus wij gingen niet naar beneden toe . Wij gingen perse niet naar beneden ."

process test fragments as corpus

In [ ]:
procs_test_term = [token for token in gensim.utils.simple_preprocess(test_term,deacc=True) if token not in dutch_stopwords]
procs_test_term_2 = [token for token in gensim.utils.simple_preprocess(test_term_2,deacc=True) if token not in dutch_stopwords]
procs_test_term_3 = [token for token in gensim.utils.simple_preprocess(test_term_3,deacc=True) if token not in dutch_stopwords]

infer vectors for test fragments

In [ ]:
inf_vector_test_term = model.infer_vector(procs_test_term)
inf_vector_test_term_2 = model.infer_vector(procs_test_term_2)
inf_vector_test_term_3 = model.infer_vector(procs_test_term_3)

The above is unstable. infer vectors as median of 100 inferral instances

In [ ]:
infarr1 = []
infarr1_cosvar = []
infarr2 = []
infarr3 = []

for i in range(100):
    inf1 = model.infer_vector(procs_test_term)
    inf1_2 = model.infer_vector(procs_test_term)
    inf2 = model.infer_vector(procs_test_term_2)
    inf3 = model.infer_vector(procs_test_term_3)
    cosvar = 1 - sp.spatial.distance.cosine(inf1,inf1_2)
    infarr1.append(inf1)
    infarr2.append(inf2)
    infarr3.append(inf3)
    infarr1_cosvar.append(cosvar)
    
med_infarr1 = np.median(np.array(infarr1),axis=0) #.mean(axis=0)
mean_infarr1 = np.array(infarr1).mean(axis=0)
std_infarr1 = np.array(infarr1).std(axis=0)
mean_infarr1_cosvar = np.array(infarr1_cosvar).mean(axis=0)
med_infarr2 = np.median(np.array(infarr2),axis=0) #.mean(axis=0)
mean_infarr2 = np.array(infarr2).mean(axis=0)
std_infarr2 = np.array(infarr2).std(axis=0)
mean_infarr3 = np.array(infarr3).mean(axis=0)
med_infarr3 = np.median(np.array(infarr3),axis=0)#.mean(axis=0)
std_infarr3 = np.array(infarr3).std(axis=0)

#print(mean_infarr1)
#print(model.infer_vector(procs_test_term,steps=100))
print(med_infarr1)
print(std_infarr1)
print(mean_infarr1_cosvar)
print('----')
print(mean_infarr3)
print(med_infarr3)
print(std_infarr3)
print('++++')
print(inf_vector_test_term_3)
print(inf_vector_test_term)

Get similar document vectors as initial test

In [ ]:
#sims = model.docvecs.most_similar([inf_vector_test_term], topn=len(model.docvecs))
#sims_2 = model.docvecs.most_similar([inf_vector_test_term_2], topn=len(model.docvecs))
#sims_3 = model.docvecs.most_similar([inf_vector_test_term_3], topn=len(model.docvecs))

sims = model.docvecs.most_similar([med_infarr1], topn=len(model.docvecs))
sims_2 = model.docvecs.most_similar([med_infarr2], topn=len(model.docvecs))
sims_3 = model.docvecs.most_similar([med_infarr3], topn=len(model.docvecs))


In [ ]:
#corp[0].words
#test = model.docvecs.similarity(corp[0].words,corp[0].words)
#print(test)
#sp.spatial.distance.cosine(test, model.docvecs[0])
#ivtt_12 = np.copy(inf_vector_test_term)
#type(inf_vector_test_term)
#model.save('/Users/eslt0101/Projects/EviDENce/ML/model_default_v50_mc2_e30')

In [ ]:
for i in range(200) :
    print(sims[i])
    if i == 19 :
        print('===================')

In [ ]:
#print(model.docvecs.similarity_unseen_docs(model,corp[0].words,corp[0].words))
#v1 = model.infer_vector(corp[0].words)
#v2 = model.infer_vector(corp[0].words)
#print(np.sqrt(1. - (sp.spatial.distance.cosine(v1,v2))**2))
#print(1. - (sp.spatial.distance.cosine(v1,v2)))
#sp.spatial.distance.cosine(mean_infarr1,model.docvecs[13946]) + 0.764286994934082

In [ ]:
for i in range(25):
    id = sims[i][0]
    print(corp_lookup[id])
    print('+++++++++++++')

reinfer corpus vectores on fully trained and froven model. these will be used for further analysis

In [ ]:

#%time reinf_corp_vec = [ model.infer_vector(c.words) for c in corp]

reinf_corp_vec_med = np.ndarray((100,len(corp),50))
i=0

for j in range(100):
    for i in range(len(corp)):
        c = corp[i]
        reinf_corp_vec_med[j][i][:] = model.infer_vector(c.words)
        i+=1
    j+=1


    
#print(aa)```````````````````````````````````````````````````````
print('=========')
print(np.median(reinf_corp_vec_med,axis=0))


In [ ]:
reinf_corp_medvec = np.median(reinf_corp_vec_med,axis=0)
print(reinf_corp_medvec)
#print(np.median(reinf_corp_vec_med,axis=0).shape  )  

In [ ]:
#len(reinf_corp_vec)
#np.save('reinferred_corpus_terms_para_150_medvec.npy',reinf_corp_medvec)

reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec.npy')



Define cosine distance function to use with set of reinferred vectors

In [ ]:
cosdist = np.array([1. - sp.spatial.distance.cosine(med_infarr1,rinfc) for rinfc in reinf_corp_medvec])

#cosdist.max()
b=np.argsort(cosdist)[::-1]
print(b[0:25])
print(cosdist[b[0:25]])
len(b)

In [ ]:
cosdist_3 = np.array([1. - sp.spatial.distance.cosine(med_infarr3,rinfc) for rinfc in reinf_corp_medvec])

#cosdist.max()
cc=np.argsort(cosdist_3)[::-1]
print(cc[0:25])
print(cosdist_3[cc[0:25]])
len(cc)

In [ ]:
def select_relevant(idlist,corp_lookup):
    selected= []
    for j in range(len(idlist)):
        jj = idlist[j]
        print('---------Snippet '+str(jj)+'---------')
        print(corp_lookup[jj])
        print('------------')
        print('Is this snippet relevant? [y/n]')
        rel = input()
        if rel == 'y' :
            selected.append(jj)
        else :
            pass
        
    return selected

In [ ]:
print('The 25 most similar snippets will be displayed.')
print(' Please select those you deem relevant. Are you ready to continue? [please select Yes/No]')
answer = input()

In [ ]:
relevant3=[]
if answer == 'Yes':
    relevant3 = select_relevant(cc[0:24],corp_lookup)
    #for i in range(25):
    #    ii = b[i]
    #    print('---------Snippet '+str(ii)+'---------')
    #    print(corp_lookup[ii])
    #    print('------------')
    #    print('Is this snippet relevant? [y/n]')
    #    rel = input()
    #    if rel == 'y' :
    #        relevant.append(ii)
    #    else :
    #        pass
else :
    print('No relevant snippets have been selected.')

The below has been run several times in an interactive fashion to select and manually iterate over the selection process. This is very muchg still in a bleeding edge development state.

In [ ]:
print(relevant3)

In [ ]:
print(len(corp_lookup))
ph =(((np.random.rand(25)*27065).round()))
print(ph)
phi = [int(pp) for pp in ph]
print(phi)

In [ ]:
relevantrand=[]
if answer == 'Yes':
    
    
    
    relevantrand = select_relevant(phi,corp_lookup)
    #for i in range(25):
    #    ii = b[i]
    #    print('---------Snippet '+str(ii)+'---------')
    #    print(corp_lookup[ii])
    #    print('------------')
    #    print('Is this snippet relevant? [y/n]')
    #    rel = input()
    #    if rel == 'y' :
    #        relevant.append(ii)
    #    else :
    #        pass
else :
    print('No relevant snippets have been selected.')

In [ ]:
print(relevantrand
     )

In [ ]:
cosdist_rel1 = np.array([1. - sp.spatial.distance.cosine(reinf_corp_medvec[20106],rinfc) for rinfc in reinf_corp_medvec])

In [ ]:
b2=np.argsort(cosdist_rel1)[::-1]
print(b2[0:25])
print(cosdist_rel1[b2[0:25]])

In [ ]:
relevant2=select_relevant(b2[0:25],corp_lookup)

#for j in range(25):
#    jj = b2[j]
#    print
#    print(corp_lookup[jj])
#    print('-------------')



In [ ]:
print(relevant2)

In [ ]:
relevant2 = [20106, 10415, 13946, 1263, 24390, 18340, 17276, 24378, 2, 4207, 20153, 17197, 25237, 5441, 8439, 26618]

In [ ]:
reit_r2 = reinf_corp_medvec[relevant2].mean(axis=0)

In [ ]:
cosdist_reit_r2 = np.array([1. - sp.spatial.distance.cosine(reit_r2,rinfc) for rinfc in reinf_corp_medvec])

In [ ]:
r2=np.argsort(cosdist_reit_r2)[::-1]
print(r2[0:25])
print(cosdist_reit_r2[r2[0:25]])

In [ ]:
relevant_r2=select_relevant(r2[0:25],corp_lookup)

In [ ]:
print(relevant_r2)

In [ ]:
reit_r3 = reinf_corp_medvec[relevant_r2].mean(axis=0)

In [ ]:
cosdist_reit_r3 = np.array([1. - sp.spatial.distance.cosine(reit_r3,rinfc) for rinfc in reinf_corp_medvec])

In [ ]:
r3=np.argsort(cosdist_reit_r3)[::-1]
print(r3[0:25])
print(cosdist_reit_r3[r3[0:25]])

In [ ]:
relevant_r3=select_relevant(r3[0:25],corp_lookup)

In [ ]:
print(relevant_r3)